# Download annual per capita nightlights 

Create a csv file with annual per capita nightlight values over DHS clusters, for years 2000 to 2013.

Import, authenticate and initialize the earth-engine library

In [1]:
import ee

ee.Authenticate()

ee.Initialize()

Enter verification code: 4/1AfJohXl4E5fa6AxZw0FPM3fPNMJtV11IHsWBfqO6QEsQHVsMe1T3aczBGxA

Successfully saved authorization token.


In [8]:
# Import other libraries
import csv
import os
import pandas as pd

Read the csv file with survey points

In [9]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/interim'
dhs_cluster_file_path = os.path.join(data_dir, 'dhs_est_iwi.csv')
df = pd.read_csv(dhs_cluster_file_path)
unique_countries = df['country'].unique()
df.head()

,country,survey_start_year,year,lat,lon,households,rural,iwi,dhs_id,image_file,...,iwi_1990_1992_est,iwi_1993_1995_est,iwi_1996_1998_est,iwi_1999_2001_est,iwi_2002_2004_est,iwi_2005_2007_est,iwi_2008_2010_est,iwi_2011_2013_est,iwi_2014_2016_est,iwi_2017_2019_est
0,south_africa,2016,2016,-34.463232,19.542468,6,1,70.723295,48830,./data/dhs_tifs/south_africa_2016/00743.tif,...,35.205078,30.981445,33.911133,43.969727,38.295898,33.579102,32.757568,38.330078,44.604492,49.267578
1,south_africa,2016,2016,-34.418873,19.188926,11,0,76.798705,48781,./data/dhs_tifs/south_africa_2016/00694.tif,...,49.243164,53.222656,56.298830,59.228516,60.986330,63.515630,66.223145,66.455080,66.137695,64.501953
2,south_africa,2016,2016,-34.412835,19.178965,4,0,81.053723,48828,./data/dhs_tifs/south_africa_2016/00741.tif,...,48.388672,51.977540,54.443360,58.715820,60.419923,63.037110,66.430664,65.934247,66.186523,64.257810
3,south_africa,2016,2016,-34.292107,19.563813,6,1,72.766880,48787,./data/dhs_tifs/south_africa_2016/00700.tif,...,21.789551,22.222900,20.300293,25.082397,27.207032,27.719727,26.947020,34.114584,36.865234,42.041016
4,south_africa,2016,2016,-34.187500,22.113079,3,0,77.864113,48756,./data/dhs_tifs/south_africa_2016/00669.tif,...,44.042970,46.875000,49.617514,48.321533,53.232420,56.865233,65.014650,65.657550,72.900390,67.529297


Function to create a feature with a 5km square around a lat/lon

In [10]:
def createDHSFeature(row):
  lat = ee.Number(row['lat'])
  lon = ee.Number(row['lon'])
  dhs_id = ee.Number(row['dhs_id'])
  properties = {'dhs_id':dhs_id,
               'lat':lat,
               'lon':lon}
    
  #Calc coordinates of a 5km bounding box around the point
  feature_radius=2500
  roi = ee.Geometry.Point(lon,lat).buffer(feature_radius).bounds()

  return ee.Feature(roi, properties)

Load nightlight and population density collections over the African continent and store in dictionary for later use

In [11]:
africa_region = ee.Geometry.Polygon([
    [-20, 38],   
    [55, 38],    
    [55, -36],   
    [-20, -36]   
])

dmsp = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').filterBounds(africa_region)
pop_count = ee.ImageCollection(ee.ImageCollection("WorldPop/GP/100m/pop"))

#get a composite image for each nighlight year and store in a year/image dictionary 
dmsp_images = {}
for year in range(2000, 2014):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Get the image and store in dictionary with year key
    img = dmsp.filterDate(start_date, end_date).select(["avg_vis"]).median()
    dmsp_images[year] = {'img':img,'year':year}

#get an image collection for each population density year and store in dictionary
pop_count_images = {}
for year in range(2000, 2014):       
    start_date = f"{year}-01-01"
    end_date = f"{year+1}-01-01"
       
    # Get imageCollection for the year (has image for each country) and store in dictionary 
    imgCol = pop_count.filterDate(start_date, end_date).select(["population"])
    pop_count_images[year] = {'imgCol':imgCol,'year':year}


Function to calculate sum of nightlights within a 5km2 square

In [12]:
def calculate_nl_sums(DHS_loc_feature):
    annual_sums = {}   #initialize empty dictionary to return
    
    #calculate annual nighlight sums
    for year, data in dmsp_images.items():
        img = data['img']
        year_value = data['year']

        nlyear = img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=DHS_loc_feature.geometry(),
            scale=927.67  #datasource pixel resolution (meters)
        )
        sum_nlyear = nlyear.getNumber('avg_vis')   #datasource band
        rounded_nlyear = ee.Number(sum_nlyear).format('%.1f')

        #append this year to the annual_sums feature set
        prop_name = f'nl{year_value}'
        annual_sums[prop_name] = rounded_nlyear 
        
    return DHS_loc_feature.set(annual_sums)

Function to calculate sum of population within a 5km2 square

In [13]:
def calculate_popcount_sums(DHS_loc_feature):
    annual_sums = {}   #initialize empty dictionary to return
        
    #calculate annual population count sums for this year
    for year, data in pop_count_images.items():
        img_collection = data['imgCol']
        year_value = data['year']     
        
        #Spatially combine country images into one mosaic image
        img = img_collection.mosaic()
       
        pcountyear = img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=DHS_loc_feature.geometry(),
                scale=92.77)    #datasource pixel resolution (meters)

        sum_pcountyear = pcountyear.getNumber('population')    #datasource band
        rounded_sum_pcountyear = ee.Number(sum_pcountyear).format('%.1f')

        #append this year to the feature set
        prop_name = f'pop_count{year_value}'
        annual_sums[prop_name] = rounded_sum_pcountyear  
    
    return DHS_loc_feature.set(annual_sums)

Get data a country at a time, write to a csv file

In [ ]:
# Prepare csv file details
csv_file_path = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/GEE/per_cap_nl_dhs_5k_WorldPop.csv'
header = ['dhs_id', 'lat', 'lon', 'nl2000', 'nl2001', 'nl2002',
         'nl2003','nl2004','nl2005','nl2006','nl2007','nl2008',
          'nl2009','nl2010','nl2011','nl2012','nl2013',
          'pop_count2000', 'pop_count2001', 'pop_count2002',
          'pop_count2003','pop_count2004','pop_count2005','pop_count2006','pop_count2007','pop_count2008',
          'pop_count2009','pop_count2010','pop_count2011','pop_count2012','pop_count2013']

# Initialize a list to store the rows for all countries
rows = []

#loop through countries, getting the data and appending to rows list
for country in unique_countries:
    country_df = df[df['country'] == country]
    print(f"Starting {country} with n rows: {len(rows)}")

    # create list of DHS locations from the country dataframe
    location_list = [createDHSFeature(row) for index, row in country_df.iterrows()]

    # Create a country-level Feature Collection from the location list
    country_collection = ee.FeatureCollection(location_list)

    # calculate nightlights and population count sums on the feature collection
    country_collection2 = country_collection.map(calculate_nl_sums)
    country_collection3 = country_collection2.map(calculate_popcount_sums)

    # Loop through the locations/features in the collection and extract the data
    for location in country_collection3.getInfo()['features']:
        properties = location['properties']
        row = [
            properties['dhs_id'],
            properties['lat'],
            properties['lon'],
            properties['nl2000'],
            properties['nl2001'],
            properties['nl2002'],
            properties['nl2003'],
            properties['nl2004'],
            properties['nl2005'],
            properties['nl2006'],
            properties['nl2007'],
            properties['nl2008'],
            properties['nl2009'],
            properties['nl2010'],
            properties['nl2011'],
            properties['nl2012'],
            properties['nl2013'],
            properties['pop_count2000'],
            properties['pop_count2001'],
            properties['pop_count2002'],
            properties['pop_count2003'],
            properties['pop_count2004'],
            properties['pop_count2005'],
            properties['pop_count2006'],
            properties['pop_count2007'],
            properties['pop_count2008'],
            properties['pop_count2009'],
            properties['pop_count2010'],
            properties['pop_count2011'],
            properties['pop_count2012'],
            properties['pop_count2013']    
        ]
        rows.append(row)
        

#Open and write to the csv file; with will close it
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)
    csv_writer.writerows(rows)

print("CSV file has been successfully written.")

Starting south_africa with n rows: 0
Starting lesotho with n rows: 746
Starting namibia with n rows: 1090
